<a href="https://colab.research.google.com/github/ricglz/CE888_activities/blob/main/assignment/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

---

In [ ]:
!pip install torch torchvision skorch timm

## Preparations

---


Before we begin, lets mount the google drive to later on read information from it:

In [ ]:
from google.colab import drive

drive_path = '/content/gdrive'
drive.mount(drive_path, force_remount=False)
drive_path += '/MyDrive'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Next we will set the seeds in everything to make this as deterministic as possible

In [ ]:
import torch
import random
import numpy as np

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

## Gather the dataset

For this we will create both our training _(which later on will be splitted into actual training an validation)_ and testing dataset.

Pytorch also allows us to have transformations like the resize and the normalization. The normalization used are [the mean and std of the ImageNet dataset](https://github.com/rwightman/pytorch-image-models/blob/master/timm/data/constants.py)

----

In [ ]:
import torchvision.transforms as T
from os import path

data_dir = path.join(drive_path, 'Essex/Datasets/Flame')
resize = T.Resize((254, 254))
normalize = T.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])

In [ ]:
train_transforms = T.Compose([
  resize,
  T.ColorJitter(brightness=0.25, contrast=0.25),
  T.RandomRotation(degrees=5),
  T.RandomHorizontalFlip(),
  T.RandomVerticalFlip(),
  T.ToTensor(),
  normalize
])
transforms = T.Compose([
  resize,
  T.ToTensor(),
  normalize
])

In [ ]:
import torchvision.datasets as datasets

train_ds = datasets.ImageFolder(path.join(data_dir, 'Training'),
                                train_transforms)
len(train_ds)

51658

In [ ]:
test_ds = datasets.ImageFolder(path.join(data_dir, 'Test'), transforms)
len(test_ds)

8617

## Create modular model 

---

In [ ]:
from torch.nn import Linear, Module
import timm

f_params = None

class PretrainedModel(Module):
    def __init__(self, model='rexnet'):
        super().__init__()
        model_name = self.get_model_name(model)
        self.model = timm.create_model(
            model_name, pretrained=True, num_classes=1)
    
    def get_model_name(self, general_model):
        return 'rexnet_200' if general_model == 'rexnet' else \
               'tf_efficientnet_b8' if general_model == 'efficientnet' else ''

    def forward(self, x):
        return self.model(x).squeeze(-1)

## Defining the API

---

### Callbacks

In this case the only Callback that will be used in every model will be an early stopping callback

In [ ]:
from skorch.callbacks import EarlyStopping, Freezer, LRScheduler, ProgressBar
from torch.optim.lr_scheduler import MultiplicativeLR, StepLR, LambdaLR

is_not_trainable = lambda x: not x.startswith('model.fc') and \
                            not x.startswith('model._fc') and \
                            not x.startswith('model.head') and \
                            not x.startswith('model.classifier')
freezer = Freezer(is_not_trainable) 
early_stopping = EarlyStopping(patience=3)
progress_bar = ProgressBar()

In [ ]:
lr = 5e-4
num_warmup_steps = 6
num_training_steps = 6
peak_lr = 8e-3

def lr_lambda(epoch):
    if epoch < num_warmup_steps:
        return (epoch / num_warmup_steps) * (peak_lr / lr)
    return max(
        0.0,
        (num_training_steps - epoch) / max(1, num_training_steps - num_warmup_steps)
    )

# lr_lambda = lambda epoch: min(2 * epoch / 10, 0.9)
scheduler = LRScheduler(policy=MultiplicativeLR, lr_lambda=lr_lambda)
# scheduler = LRScheduler(policy=StepLR, gamma=8.95e-1, step_size=1)

### Classifier class

In [ ]:
from torch import float64
from skorch.classifier import NeuralNetBinaryClassifier
from skorch.utils import to_tensor, to_numpy
import sklearn.metrics as sk_metrics 
import numpy as np

class MyClassifier(NeuralNetBinaryClassifier):
    def infer(self, x, **fit_params):
        x = to_tensor(x, device=self.device)
        if isinstance(x, dict):
            x_dict = self._merge_x_and_fit_params(x, fit_params)
            return self.module_(**x_dict).to(device=self.device, dtype=float64)
        return self.module_(x, **fit_params).to(device=self.device, dtype=float64)

    def train_step_single(self, Xi, yi, **fit_params):
        self.module_.train()
        y_pred = self.infer(Xi, **fit_params)
        yi = yi.to(device=self.device, dtype=float64)
        loss = self.get_loss(y_pred, yi, X=Xi, training=True)
        loss.backward()
        return { 'loss': loss, 'y_pred': y_pred }

    def validation_step(self, Xi, yi, **fit_params):
        self.module_.eval()
        y_pred = self.infer(Xi, **fit_params)
        yi = yi.to(device=self.device, dtype=float64)
        loss = self.get_loss(y_pred, yi, X=Xi, training=False)
        return { 'loss': loss,'y_pred': y_pred }

    def _get_y_values(self, X):
        y_true, y_pred = [], []
        nonlinearity = self._get_predict_nonlinearity()
        for images, labels in self.get_iterator(X):
            images = images.to(self.device)
            outputs = nonlinearity(self.module_(images))
            _, predicted = torch.max(outputs.data, 1)
            y_true.append(to_numpy(labels))
            y_pred.append(to_numpy(predicted))
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        return y_true, y_pred

    def score(self, X):
        y_true, y_pred = self._get_y_values(X)
        return sk_metrics.roc_auc_score(y_true, y_pred)
    
    def scores(self, X):
        y_true, y_pred = self._get_y_values(X)
        accuracy = sk_metrics.accuracy_score(y_true, y_pred)
        confusion_matrix = sk_metrics.confusion_matrix(y_true, y_pred)
        f1 = sk_metrics.f1_score(y_true, y_pred)
        auc = sk_metrics.roc_auc_score(y_true, y_pred)
        return accuracy, confusion_matrix, f1, auc

    def print_and_plot_scores(self, X):
        accuracy, confusion_matrix, f1, auc = self.scores(X)
        print(f'Accuracy: {accuracy}')
        print(f'F1 Score: {f1}')
        print(f'AUC: {auc}')
        disp = sk_metrics.ConfusionMatrixDisplay(
          confusion_matrix, display_labels=['Fire', 'No_Fire'])
        disp.plot()

### Classifier helper functions

The next code will be used to create helper functions to easily create, fit and evaluate different type of CNN architectures

In [ ]:
from torch.optim import Adam
from skorch.callbacks import Checkpoint
from skorch.dataset import CVSplit
from skorch.helper import predefined_split

def create_model(module_model):
    global f_params

    f_params = path.join(drive_path, f'Models/best_{module_model}.pt')
    checkpoint = Checkpoint(f_params=f_params, monitor='valid_acc_best')
    callbacks = [checkpoint, freezer, scheduler]

    return MyClassifier(
        PretrainedModel,
        module__model=module_model,
        optimizer=Adam,
        lr=lr,
        batch_size=64,
        max_epochs=15,
        iterator_train__shuffle=True,
        iterator_train__num_workers=16,
        iterator_valid__shuffle=True,
        iterator_valid__num_workers=16,
        train_split=CVSplit(0.2, random_state=seed),
        callbacks=callbacks,
        device='cuda'
    )

In [ ]:
def create_and_fit(model_name):
    net = create_model(model_name)
    net.fit(train_ds, y=None)
    net.load_params(f_params=f_params)
    return net

## Models results

---

### Rexnet

In [ ]:
rexnet = create_and_fit('rexnet')

  epoch    train_loss    valid_acc    valid_loss    cp      lr       dur
-------  ------------  -----------  ------------  ----  ------  --------
      1        0.6338       0.6574        1.6776     +  0.0005  334.9860


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


      2        0.1727       0.6929        1.1489     +  0.0027  338.9241
      3        0.0610       0.7809        1.0547     +  0.0284  337.1139
      4        0.0450       0.7203        1.2561        0.0284  338.2943
      5        0.0431       0.7311        1.4398        0.0253  337.9582
      6        0.0401       0.6529        2.0123        0.0197  337.3363
      7        0.0335       0.6798        2.1060        0.0131  338.0526
      8        0.0333       0.6863        1.7308        0.0073  341.2416
      9        0.0255       0.6951        1.4303        0.0032  343.8477
     10        0.0218       0.7011        1.3842        0.0011  338.7790
     11        0.0259       0.7071        1.3625        0.0002  336.2726


In [ ]:
rexnet.print_and_plot_scores(test_ds)

### EfficientNet

In [ ]:
efficientnet = create_and_fit('efficientnet')

In [ ]:
efficientnet.print_and_plot_scores(test_ds)